# Generate HTML

This notebook generates the HTML files for the dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path

from depsurf import DepKind, VersionGroup, DepReport, Dep
from utils import WEBSITE_PATH


GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]


def get_report_path(dep: Dep) -> Path:
    return (
        WEBSITE_PATH
        / dep.kind
        / dep.name.replace("_", "").lower()[0]
        / f"{dep.name}.json"
    )


DEPS = [
    DepKind.FUNC("__blk_account_io_done"),
    DepKind.FUNC("__page_cache_release"),
    DepKind.FUNC("vfs_read"),
    DepKind.STRUCT("task_struct"),
    DepKind.STRUCT("fs_context"),
    DepKind.STRUCT("inode"),
]

# for dep in DEPS:
#     report = DepReport.from_groups(dep, GROUPS)
#     report.dump(get_report_path(dep))

# DepReport.from_dump(get_report_path(DEPS[0]))


In [3]:
FUNC_DEPS = sorted(
    set(
        DepKind.FUNC(name)
        for group in GROUPS
        for v in group
        for name in v.img.get_all_by_kind(DepKind.FUNC).keys()
    )
)

FUNC_REPORTS = [DepReport.from_groups(dep, GROUPS) for dep in FUNC_DEPS]

[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.4.0-21-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.8.0-22-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.10.0-19-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.13.0-16-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.15.0-20-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/4.18.0-10-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/5.0.0-13-generic-amd64.jsonl
[        types.py:19 ] INFO: Loading types from /Users/szhong/Code/DepSurf/data/dataset/types_func/5

In [4]:
for report in FUNC_REPORTS:
    report.dump(get_report_path(report.dep))


In [5]:
from depsurf import IssueEnum, VersionGroup

from collections import defaultdict

partitioned_results = defaultdict(dict)

for report in FUNC_REPORTS:
    result = {}
    for (group, version), issues in report.issue_dict.items():
        v = version.short_version
        if IssueEnum.FULL_INLINE in issues:
            result[v] = "F"
        elif IssueEnum.SELECTIVE_INLINE in issues:
            result[v] = "S"
        elif IssueEnum.ABSENT in issues:
            result[v] = "-"
        else:
            result[v] = ""

    dep = report.dep
    key = dep.name.replace("_", "").lower()
    partitioned_results[key[:1]][(key, dep.name)] = result


partitioned_results = {k: sorted(v.items()) for k, v in partitioned_results.items()}

In [6]:
INLINED_PATH = WEBSITE_PATH / "inline"
INLINED_PATH.mkdir(exist_ok=True)

for char, results in partitioned_results.items():
    with open(INLINED_PATH / f"{char}.html", "w") as f:
        f.write("<html>\n")
        f.write("<head>\n")
        f.write(f"<title>Kernel Function Inline Status - {char}</title>\n")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write("</head>\n")

        f.write("<body>\n")
        f.write("<table>\n")
        f.write("<thead>\n")

        f.write("<tr>\n")
        f.write(
            f'<td><span class="f">&nbsp;F&nbsp;</span>: Fully Inlined. <span class="p">&nbsp;P&nbsp;</span>: Partially Inlined. <span class="m">&nbsp;-&nbsp;</span>: Absent</td>\n'
        )
        for version in VersionGroup.REGULAR:
            f.write(f"<th>{version.short_version}</th>")
        f.write("</tr>\n")
        f.write("</thead>\n")

        f.write("<tbody>\n")
        for (_, dep), result in results:
            f.write(f'<tr><th id="{dep}"><pre>{dep}</pre></th>')
            for v in result.values():
                class_name = {
                    "F": "f",
                    "P": "p",
                    "-": "m",
                }.get(v)
                if class_name is not None:
                    f.write(f'<td class="{class_name}">{v}</td>')
                else:
                    f.write(f"<td>{v}</td>")
            f.write("</tr>\n")
        f.write("</tbody>\n")

        f.write("</table>\n")
        f.write("</body>\n")
        f.write("</html>\n")